In [1]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Lambda

Using TensorFlow backend.


In [2]:
data_dict = pickle.load(open('../data/fitzgerald/processed-all-books.pickle', 'rb'))

In [3]:
X = data_dict['X']
Y = data_dict['Y']
dataX = data_dict['dataX']
dataY = data_dict['dataY']
char_to_idx = data_dict['char_to_idx']
idx_to_char = data_dict['idx_to_char']

# Constants
m, n_timesteps, _ = X.shape
_, n_chars = Y.shape
n_a = 64

In [4]:
print('X shape:', X.shape)
print('Y shape:', Y.shape)
print('Num. Timesteps:', n_timesteps)
print('Num. Unique Chars:', n_chars)

X shape: (2029021, 140, 1)
Y shape: (2029021, 67)
Num. Timesteps: 140
Num. Unique Chars: 67


In [5]:
def get_model(n_timesteps, n_features, n_a):
    x0 = Input(shape=(n_timesteps,1), name='input')
    
    X = LSTM(n_a, return_sequences=True)(x0)
    X = LSTM(n_a)(X)
    out = Dense(n_features, activation='softmax')(X)
    model = Model(x0, out)
    
    return model

In [6]:
model = get_model(n_timesteps, n_chars, n_a)

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [8]:
a_initial = np.zeros((n_timesteps, n_a))
c_initial = np.zeros((n_timesteps, n_a))

In [30]:
history = model.fit(X, Y, 
                    validation_split=0.2,
                    epochs=20, 
                    batch_size=1024, 
                    verbose=1)

Train on 1623216 samples, validate on 405805 samples
Epoch 1/20
1623216/1623216 [==============================] - 360s 222us/step - loss: 2.2718 - val_loss: 2.4016
Epoch 2/20
1623216/1623216 [==============================] - 356s 219us/step - loss: 2.2643 - val_loss: 2.3880
Epoch 3/20
1623216/1623216 [==============================] - 355s 219us/step - loss: 2.2574 - val_loss: 2.3911
Epoch 4/20
1623216/1623216 [==============================] - 356s 219us/step - loss: 2.2508 - val_loss: 2.3805
Epoch 5/20
1623216/1623216 [==============================] - 356s 219us/step - loss: 2.2444 - val_loss: 2.3779
Epoch 6/20
1623216/1623216 [==============================] - 356s 219us/step - loss: 2.2386 - val_loss: 2.3776
Epoch 7/20
1623216/1623216 [==============================] - 356s 219us/step - loss: 2.2328 - val_loss: 2.3729
Epoch 8/20
1623216/1623216 [==============================] - 356s 219us/step - loss: 2.2271 - val_loss: 2.3610
Epoch 9/20
1623216/1623216 [=======================

In [41]:
num_epochs = 42
model.save('../data/trained-fitzgerald-{}-epochs'.format(num_epochs))

In [38]:
# Generate Text
# Given seed sentence - 100 characters - predict next character
# For every predicted char, add to predictions = []

def reshape_input(original_input):
    return np.reshape(original_input, (1, n_timesteps, 1))

def get_p_idx(p):
    flattened = np.ndarray.flatten(np.array(p))
    return np.random.choice([i for i in range(n_chars)], 
                            p = flattened)

def generate_text(seed_input, model, num_chars_to_generate=140):
    if len(seed_input) < 100:
        raise Exception('Seed_input must be at least 140 characters')
    curr_input = seed_input.lower()
    curr_input = list(curr_input)
    curr_input = curr_input[:n_timesteps] # first 100 chars
    
    curr_input = [char_to_idx[c] for c in curr_input]
    # Normalize
    curr_input = np.array(curr_input) / n_chars
    predictions = []
    
    for i in range(num_chars_to_generate):
        p = model.predict(reshape_input(curr_input))
        p_idx = get_p_idx(p)
        predictions.append(idx_to_char[p_idx])
        curr_input = np.append(curr_input[1:], p_idx / n_chars)
    
    return ''.join(predictions)

In [39]:
seed_input = "Then abroad again--to Rome this time, where he dallied with architecture and painting in turn, took up the violin, and wrote some ghastly Italian sonnets, supposedly the ruminations of a thirteenth-century monk on the joys of the contemplative life. It became established among his Harvard intimates that he was in Rome, and those of them who were abroad that year looked him up and discovered with him, on many moonlight excursions, much in the city that was older than the Renaissance or indeed than the republic. Maury Noble, from Philadelphia, for instance, remained two months, and together they realized the peculiar charm of Latin women and had a delightful sense of being very young and free in a civilization that was very old and free. Not a few acquaintances of his grandfather's called on him, and had he so desired he might have been _persona grata_ with the diplomatic set--indeed, he found that his inclinations tended more and more toward conviviality, but that long adolescent aloofness and consequent shyness still dictated to his conduct."

In [40]:
generate_text(seed_input, model)

' heddny,"shnukng agoiy and cxer crbissalt soatioing vhar fasosned. "were ceeu i wo dlmver\'  she copagoy.\n""pooeer\'  buai!ttraknly, in grmuor'